## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [3]:
import tensorflow as tf

In [2]:
#!pip install tensorflow

In [3]:
tf.enable_eager_execution()

In [4]:
tf.executing_eagerly()

True

In [5]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [6]:
#from google.colab import drive
#drive.mount('/gdrive')

In [7]:
import pandas as pd

In [8]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


In [10]:
data.describe()

,open,close,low,high,volume
count,851264.000000,851264.000000,851264.000000,851264.000000,8.512640e+05
mean,70.836986,70.857109,70.118414,71.543476,5.415113e+06
std,83.695876,83.689686,82.877294,84.465504,1.249468e+07
min,0.850000,0.860000,0.830000,0.880000,0.000000e+00
25%,33.840000,33.849998,33.480000,34.189999,1.221500e+06
50%,52.770000,52.799999,52.230000,53.310001,2.476250e+06
75%,79.879997,79.889999,79.110001,80.610001,5.222500e+06
max,1584.439941,1578.130005,1549.939941,1600.930054,8.596434e+08


### Drop columns `date` and  `symbol`

In [11]:
del data['date']

In [12]:
del data['symbol']

In [13]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [14]:
data1 = data.head(1000)

In [15]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
data1['volume'] = data['volume'].div(1000)

In [17]:
data1.shape

(1000, 5)

In [18]:
data1.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163.6
1,125.239998,119.980003,119.940002,125.540001,2386.4
2,116.379997,114.949997,114.930000,119.739998,2489.5
3,115.480003,116.620003,113.500000,117.440002,2006.3
4,117.010002,114.970001,114.089996,117.330002,1408.6


### Divide the data into train and test sets

In [19]:
from sklearn.model_selection import train_test_split
X = data1.iloc[:,0:4]
Y = data1.iloc[:,4]
# split into 70% for train and 30% for test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [20]:
y_test.shape

(300,)

In [21]:
Y.shape

(1000,)

#### Convert Training and Test Data to numpy float32 arrays


### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [22]:
from sklearn.preprocessing import Normalizer
from sklearn import preprocessing

In [23]:
#Normalize the data
x_n_train = preprocessing.normalize(X_train)
x_n_test = preprocessing.normalize(X_test)

In [24]:
x_n_train = x_n_train.astype('float32')
x_n_test = x_n_test.astype('float32')

In [25]:
import numpy as np

In [26]:
y_train = np.array(y_train.astype('float32'))
y_test = np.array(y_test.astype('float32'))

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [27]:
w = tf.random_normal((4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [28]:
# y = wx + b

def prediction(x, w, b):
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    return y

3.Loss (Cost) Function [Mean square error]

In [29]:
#Define Loss
def loss(y_actual, y_predicted):
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [30]:
#Define Gradient Descent function
def train(x, y_actual, w, b, learning_rate=0.01):
    #Record math ops on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    #Calculate Gradients for Loss w.r.t Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Test loss at every 5th iteration

In [31]:
y_test.shape

(300,)

In [32]:
#Training the model for 100 iterations
for i in range(100):
    w, b = train(x_n_train, y_train, w, b)
    print('Current Loss on iteration', i, loss(y_train, prediction(x_n_train, w, b)).numpy())

Current Loss on iteration 0 223484620.0
Current Loss on iteration 1 221283060.0
Current Loss on iteration 2 219254240.0
Current Loss on iteration 3 217384270.0
Current Loss on iteration 4 215661040.0
Current Loss on iteration 5 214072900.0
Current Loss on iteration 6 212609390.0
Current Loss on iteration 7 211260590.0
Current Loss on iteration 8 210017230.0
Current Loss on iteration 9 208871620.0
Current Loss on iteration 10 207815710.0
Current Loss on iteration 11 206842640.0
Current Loss on iteration 12 205945870.0
Current Loss on iteration 13 205119280.0
Current Loss on iteration 14 204357760.0
Current Loss on iteration 15 203655630.0
Current Loss on iteration 16 203008720.0
Current Loss on iteration 17 202412530.0
Current Loss on iteration 18 201862880.0
Current Loss on iteration 19 201356460.0
Current Loss on iteration 20 200889920.0
Current Loss on iteration 21 200459760.0
Current Loss on iteration 22 200063410.0
Current Loss on iteration 23 199698000.0
Current Loss on iteration 

In [33]:
#Testing the model for 100 iterations and printing every 5 iteration
for i in range(100):
    w, b = train(x_n_test, y_test, w, b)
    if i % 5 == 0:
        print ('Test loss at every 5th step: ', i, ' is ', loss(y_test, prediction(x_n_test, w, b)).numpy())

Test loss at every 5th step:  0  is  240872350.0
Test loss at every 5th step:  5  is  240763280.0
Test loss at every 5th step:  10  is  240690740.0
Test loss at every 5th step:  15  is  240642320.0
Test loss at every 5th step:  20  is  240610350.0
Test loss at every 5th step:  25  is  240589150.0
Test loss at every 5th step:  30  is  240574780.0
Test loss at every 5th step:  35  is  240565280.0
Test loss at every 5th step:  40  is  240558940.0
Test loss at every 5th step:  45  is  240555100.0
Test loss at every 5th step:  50  is  240552200.0
Test loss at every 5th step:  55  is  240550270.0
Test loss at every 5th step:  60  is  240548980.0
Test loss at every 5th step:  65  is  240548220.0
Test loss at every 5th step:  70  is  240547760.0
Test loss at every 5th step:  75  is  240547440.0
Test loss at every 5th step:  80  is  240547100.0
Test loss at every 5th step:  85  is  240546880.0
Test loss at every 5th step:  90  is  240546930.0
Test loss at every 5th step:  95  is  240546830.0


### Get the shapes and values of W and b

In [34]:
w.shape

TensorShape([Dimension(4), Dimension(1)])

In [35]:
w

<tf.Tensor: id=9686, shape=(4, 1), dtype=float32, numpy=
array([[1208.9374],
       [1213.0725],
       [1198.5361],
       [1222.784 ]], dtype=float32)>

In [36]:
b.shape

TensorShape([Dimension(1)])

In [37]:
b

<tf.Tensor: id=9689, shape=(1,), dtype=float32, numpy=array([2421.2363], dtype=float32)>

### Model Prediction on 1st Examples in Test Dataset

In [38]:
X_value = X_test.head(1)
X_value= X_value.astype('float32')

In [39]:
X_value=X_value.values

In [40]:
prediction(X_value, w, b)

<tf.Tensor: id=9694, shape=(1, 1), dtype=float32, numpy=array([[428522.8]], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load tensorflow if not done already

### Load the given Iris data using pandas (Iris.csv)

In [4]:
from keras.models import Sequential
from keras.layers import Dense

In [5]:
tf.disable_eager_execution()

In [6]:
tf.executing_eagerly()

False

In [8]:
import numpy as np
import pandas as pd

In [9]:
data_iris = pd.read_csv('Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [10]:
df_with_dummies = pd.get_dummies(data_iris, columns = ['Species'])

In [16]:
df_with_dummies.head(3)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0


### Splitting the data into feature set and target set

In [17]:
del df_with_dummies['Id']

In [18]:
df_with_dummies.head(3)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0


In [29]:
from sklearn.model_selection import train_test_split
X = df_with_dummies.iloc[:,0:4]
Y = df_with_dummies.iloc[:,4:7]

### Divide the dataset into Training and test (70:30)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=1)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [49]:
#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(3,activation='softmax'))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Model Training 

In [50]:
model.fit(X_train, y_train,validation_data=(X_test, y_test),epochs=50,batch_size=5)

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 1s 6ms/sample - loss: 1.9689 - acc: 0.1714 - val_loss: 3.2241 - val_acc: 0.2444
Epoch 2/50
105/105 [==============================] - 0s 604us/sample - loss: 1.5604 - acc: 0.2667 - val_loss: 2.6995 - val_acc: 0.2889
Epoch 3/50
105/105 [==============================] - 0s 851us/sample - loss: 1.3535 - acc: 0.2667 - val_loss: 2.3015 - val_acc: 0.2889
Epoch 4/50
105/105 [==============================] - 0s 922us/sample - loss: 1.1825 - acc: 0.3048 - val_loss: 1.9635 - val_acc: 0.2889
Epoch 5/50
105/105 [==============================] - 0s 903us/sample - loss: 1.0630 - acc: 0.3143 - val_loss: 1.6819 - val_acc: 0.2889
Epoch 6/50
105/105 [==============================] - 0s 780us/sample - loss: 0.9358 - acc: 0.3143 - val_loss: 1.4478 - val_acc: 0.2889
Epoch 7/50
105/105 [==============================] - 0s 998us/sample - loss: 0.8459 - acc: 0.6190 - val_loss: 1.2548 - val_acc: 0.3333
Epoch

### Model Prediction

In [51]:
y_predict = model.predict_classes(X_test)

In [52]:
y_predict

array([0, 1, 1, 0, 2, 2, 2, 0, 0, 2, 1, 0, 2, 2, 2, 0, 1, 2, 0, 0, 1, 2,
       2, 0, 2, 1, 0, 0, 1, 2, 2, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2,
       1], dtype=int64)

### Save the Model

In [57]:
model.save('saved_model.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?